# CIS 678 - Machine Learning - Kaggle Challenge 3: MNIST Digit Classification

## Data Loading

## Model Implementation

## Model Evaluation

## Conclusion

In [1]:
import numpy as np

print('hello world!')

hello world!
